# Phonon computations

This is a quick sketch how to run a simple phonon calculation using DFTK.

> **Preliminary implementation**
>
> Practical phonon computations have only seen rudimentary testing as of now.
> As of now we do not yet recommend relying on this feature for production
> calculations. We appreciate any issues, bug reports or PRs.

First we run an SCF calculation.

In [1]:
using AtomsBuilder
using DFTK
using PseudoPotentialData

pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
model  = model_DFT(bulk(:Si); pseudopotentials, functionals=LDA())
basis  = PlaneWaveBasis(model; Ecut=10, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);
nothing  # hide

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.916474679381                   -0.69    4.8    109ms
  2   -7.921208160538       -2.32       -1.52    1.0   79.4ms
  3   -7.921403808925       -3.71       -2.49    1.4   59.1ms
  4   -7.921440850717       -4.43       -2.81    2.8   78.9ms
  5   -7.921441542246       -6.16       -2.95    1.0   55.7ms
  6   -7.921442006228       -6.33       -4.45    1.0   55.9ms
  7   -7.921442021671       -7.81       -4.53    2.8    100ms
  8   -7.921442022127       -9.34       -5.19    1.0   56.8ms
  9   -7.921442022142      -10.82       -5.57    1.2   59.0ms
 10   -7.921442022143      -12.10       -5.69    1.5   62.2ms
 11   -7.921442022144      -11.91       -6.96    1.0    280ms
 12   -7.921442022144      -13.15       -6.95    2.8   93.8ms
 13   -7.921442022144      -14.75       -7.78    1.0   57.1ms
 14   -7.921442022144   +    -Inf       -8.01    1.2   60.9ms


Next we compute the phonon modes at the q-point `[1/4, 1/4, 1/4]`.

In [2]:
scfres = DFTK.unfold_bz(scfres)
phret_q0 = @time DFTK.phonon_modes(scfres; q=[0.25, 0.25, 0.25]);
nothing  # hide

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      73.0   2.59s  Non-interacting
   1        0       1        0.31     52.6   9.17s  
   2        0       2       -0.52     47.0   1.59s  
   3        0       3       -2.40     41.8   1.37s  
   4        0       4       -4.46     29.7   1.10s  
   5        0       5       -6.46     17.5   783ms  
   6        0       6       -8.91      4.0   429ms  
   7        1       1       -7.81     55.9   2.78s  Restart
   8        1       2       -8.90      5.6   467ms  
                                      72.8   2.04s  Final orbitals
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      73.0   1.89s  Non-interacting
   1        0       1        0.31     52.6   1.74s  
   2        0       2       -0.52     47.0   1.50s  
   3

These are the final phonon frequencies:

In [3]:
phret_q0.frequencies

6-element Vector{Float64}:
 -0.0034981553511830993
 -0.0029774511552584115
 -0.0029774511548889574
  0.004302230648874873
  0.004302230648972435
  0.004353201776256149